# Chicago Car Crash Analysis

## Importing Packages

In [192]:
# Importing packages needed for this notebook

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.impute import SimpleImputer

In [193]:
# Reading the data in

df_crashes = pd.read_csv("data/Traffic_Crashes_-_Crashes.csv")
df_vehicles = pd.read_csv("data/Traffic_Crashes_-_Vehicles.csv")
df_people = pd.read_csv("data/Traffic_Crashes_-_People.csv")

/var/folders/kl/l56g5k7x0xqgjd17r78jnyxc0000gn/T/ipykernel_21831/2665594103.py:4: DtypeWarning: Columns (2,19,21,40,41,42,44,48,49,50,53,55,58,59,61,71) have mixed types. Specify dtype option on import or set low_memory=False.
  df_vehicles = pd.read_csv("data/Traffic_Crashes_-_Vehicles.csv")
/var/folders/kl/l56g5k7x0xqgjd17r78jnyxc0000gn/T/ipykernel_21831/2665594103.py:5: DtypeWarning: Columns (29) have mixed types. Specify dtype option on import or set low_memory=False.
  df_people = pd.read_csv("data/Traffic_Crashes_-_People.csv")


In [194]:
# Dropping columns

columns_to_drop_crashes = ['RD_NO', 'CRASH_DATE_EST_I', 'LANE_CNT', 'REPORT_TYPE', 'INTERSECTION_RELATED_I', 'NOT_RIGHT_OF_WAY_I', 'HIT_AND_RUN_I', 'DATE_POLICE_NOTIFIED',
                           'SEC_CONTRIBUTORY_CAUSE', 'STREET_NO', 'STREET_DIRECTION', 'STREET_NAME', 'BEAT_OF_OCCURRENCE', 'PHOTOS_TAKEN_I', 'STATEMENTS_TAKEN_I', 'DOORING_I',
                           'WORK_ZONE_I', 'WORK_ZONE_TYPE', 'WORKERS_PRESENT_I', 'INJURIES_FATAL', 'INJURIES_INCAPACITATING', 'INJURIES_NON_INCAPACITATING',
                           'INJURIES_REPORTED_NOT_EVIDENT','INJURIES_NO_INDICATION', 'INJURIES_UNKNOWN', 'LATITUDE', 'LONGITUDE', 'LOCATION', 'Boundaries - ZIP Codes'
                           ]

columns_to_drop_vehicles = ['RD_NO', 'CRASH_DATE', 'CRASH_UNIT_ID', 'UNIT_NO', 'UNIT_TYPE', 'NUM_PASSENGERS', 'VEHICLE_ID', 'CMRC_VEH_I', 'MAKE', 'MODEL', 'VEHICLE_YEAR',
                            'VEHICLE_DEFECT','VEHICLE_USE', 'TRAVEL_DIRECTION', 'TOWED_I', 'FIRE_I', 'OCCUPANT_CNT', 'EXCEED_SPEED_LIMIT_I', 'TOWED_BY', 'TOWED_TO',
                            'AREA_00_I', 'AREA_01_I', 'AREA_02_I', 'AREA_03_I', 'AREA_04_I', 'AREA_05_I', 'AREA_06_I', 'AREA_07_I', 'AREA_08_I', 'AREA_09_I',
                            'AREA_10_I', 'AREA_11_I', 'AREA_12_I', 'AREA_99_I',  'CMV_ID', 'USDOT_NO', 'CCMC_NO', 'ILCC_NO', 'COMMERCIAL_SRC',
                            'GVWR', 'CARRIER_NAME', 'CARRIER_STATE', 'CARRIER_CITY', 'HAZMAT_PLACARDS_I', 'HAZMAT_NAME','UN_NO', 'HAZMAT_PRESENT_I',
                            'HAZMAT_REPORT_I', 'HAZMAT_REPORT_NO', 'MCS_REPORT_I', 'MCS_REPORT_NO', 'HAZMAT_VIO_CAUSE_CRASH_I', 'MCS_VIO_CAUSE_CRASH_I', 'IDOT_PERMIT_NO',
                            'WIDE_LOAD_I', 'TRAILER1_WIDTH', 'TRAILER2_WIDTH', 'TRAILER1_LENGTH', 'TRAILER2_LENGTH', 'TOTAL_VEHICLE_LENGTH', 'AXLE_CNT', 'VEHICLE_CONFIG', 'CARGO_BODY_TYPE',
                            'LOAD_TYPE', 'HAZMAT_OUT_OF_SERVICE_I', 'MCS_OUT_OF_SERVICE_I', 'HAZMAT_CLASS'
                            ]

columns_to_drop_people = ['RD_NO', 'CRASH_DATE', 'VEHICLE_ID', 'PERSON_ID', 'SEAT_NO', 'CITY', 'STATE', 'ZIPCODE', 'DRIVERS_LICENSE_STATE', 'DRIVERS_LICENSE_CLASS', 'EJECTION',
                          'HOSPITAL', 'EMS_AGENCY', 'EMS_RUN_NO', 'BAC_RESULT VALUE', 'PEDPEDAL_ACTION', 'PEDPEDAL_VISIBILITY', 'PEDPEDAL_LOCATION', 'CELL_PHONE_USE', 'INJURY_CLASSIFICATION'
                          ]

df_crashes = df_crashes.drop(columns_to_drop_crashes, axis=1)
df_vehicles = df_vehicles.drop(columns_to_drop_vehicles, axis=1)
df_people = df_people.drop(columns_to_drop_people, axis=1)

## Creating a Master DataFrame

In [195]:
master_df = pd.merge(df_crashes, pd.merge(df_vehicles, df_people, on="CRASH_RECORD_ID"), on="CRASH_RECORD_ID")

## Info and column names of each table

In [196]:
master_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3594423 entries, 0 to 3594422
Data columns (total 34 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   CRASH_RECORD_ID          object 
 1   CRASH_DATE               object 
 2   POSTED_SPEED_LIMIT       int64  
 3   TRAFFIC_CONTROL_DEVICE   object 
 4   DEVICE_CONDITION         object 
 5   WEATHER_CONDITION        object 
 6   LIGHTING_CONDITION       object 
 7   FIRST_CRASH_TYPE         object 
 8   TRAFFICWAY_TYPE          object 
 9   ALIGNMENT                object 
 10  ROADWAY_SURFACE_COND     object 
 11  ROAD_DEFECT              object 
 12  CRASH_TYPE               object 
 13  DAMAGE                   object 
 14  PRIM_CONTRIBUTORY_CAUSE  object 
 15  NUM_UNITS                int64  
 16  MOST_SEVERE_INJURY       object 
 17  INJURIES_TOTAL           float64
 18  CRASH_HOUR               int64  
 19  CRASH_DAY_OF_WEEK        int64  
 20  CRASH_MONTH              int64  
 21  LIC_PLAT

## Takeaway

- 3,594,423 observations
- 36 combined columns
- 987 MB combined memory file

## Data Cleaning

In [168]:
# Filtering from 2020 and on

master_df['CRASH_DATE'] = pd.to_datetime(master_df['CRASH_DATE'])
master_df['YEAR'] = master_df['CRASH_DATE'].dt.year
master_df = master_df.loc[master_df['INJURIES_TOTAL'] > 0]
master_df = master_df[
    (master_df["YEAR"] >= 2020) &
    ((master_df["VEHICLE_TYPE"] == "PASSENGER") |
     (master_df["VEHICLE_TYPE"] == "SPORT UTILITY VEHICLE (SUV)") |
     (master_df["VEHICLE_TYPE"] == "VAN/MINI-VAN"))
]
master_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 305598 entries, 14 to 1858292
Data columns (total 36 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   CRASH_RECORD_ID          305598 non-null  object        
 1   CRASH_DATE               305598 non-null  datetime64[ns]
 2   POSTED_SPEED_LIMIT       305598 non-null  int64         
 3   TRAFFIC_CONTROL_DEVICE   305598 non-null  object        
 4   DEVICE_CONDITION         305598 non-null  object        
 5   WEATHER_CONDITION        305598 non-null  object        
 6   LIGHTING_CONDITION       305598 non-null  object        
 7   FIRST_CRASH_TYPE         305598 non-null  object        
 8   TRAFFICWAY_TYPE          305598 non-null  object        
 9   ALIGNMENT                305598 non-null  object        
 10  ROADWAY_SURFACE_COND     305598 non-null  object        
 11  ROAD_DEFECT              305598 non-null  object        
 12  CRASH_TYPE         

## Takeaway

- We filtered the data from year 2020 and after, 

In [169]:
master_df.isna().sum()

CRASH_RECORD_ID                0
CRASH_DATE                     0
POSTED_SPEED_LIMIT             0
TRAFFIC_CONTROL_DEVICE         0
DEVICE_CONDITION               0
WEATHER_CONDITION              0
LIGHTING_CONDITION             0
FIRST_CRASH_TYPE               0
TRAFFICWAY_TYPE                0
ALIGNMENT                      0
ROADWAY_SURFACE_COND           0
ROAD_DEFECT                    0
CRASH_TYPE                     0
DAMAGE                         0
PRIM_CONTRIBUTORY_CAUSE        0
NUM_UNITS                      0
MOST_SEVERE_INJURY             0
INJURIES_TOTAL                 0
CRASH_HOUR                     0
CRASH_DAY_OF_WEEK              0
CRASH_MONTH                    0
LIC_PLATE_STATE            12230
VEHICLE_TYPE                   0
MANEUVER                       0
FIRST_CONTACT_POINT           86
PERSON_TYPE                    0
SEX                         7393
AGE                        53949
SAFETY_EQUIPMENT            1633
AIRBAG_DEPLOYED            11910
INJURY_CLA

# Andy's Space

# Harshitha's Space

# Dre's Space

In [183]:
master_df['DRIVER_ACTION'].value_counts(normalize=True)

DRIVER_ACTION
NONE                                 0.392392
UNKNOWN                              0.210209
FAILED TO YIELD                      0.111539
OTHER                                0.101224
DISREGARDED CONTROL DEVICES          0.049397
FOLLOWED TOO CLOSELY                 0.044941
TOO FAST FOR CONDITIONS              0.023470
IMPROPER TURN                        0.022609
IMPROPER LANE CHANGE                 0.011950
IMPROPER PASSING                     0.009679
WRONG WAY/SIDE                       0.008765
IMPROPER BACKING                     0.004261
OVERCORRECTED                        0.002893
EVADING POLICE VEHICLE               0.002817
CELL PHONE USE OTHER THAN TEXTING    0.001516
EMERGENCY VEHICLE ON CALL            0.001473
IMPROPER PARKING                     0.000454
TEXTING                              0.000311
LICENSE RESTRICTIONS                 0.000081
STOPPED SCHOOL BUS                   0.000019
Name: proportion, dtype: float64

## Consideration

- use "NONE" for the null values.

In [187]:
imputer - SimpleImputer(strategy='mode')

imputer.fit(master_df['DRIVER_VISION'])



master_df['DRIVER_VISION'].value_counts()

DRIVER_VISION
NOT OBSCURED              105839
UNKNOWN                    96639
OTHER                       2541
MOVING VEHICLES             1713
PARKED VEHICLES              927
WINDSHIELD (WATER/ICE)       559
BLINDED - SUNLIGHT           427
TREES, PLANTS                 90
BUILDINGS                     79
BLINDED - HEADLIGHTS          43
BLOWING MATERIALS             24
HILLCREST                     22
EMBANKMENT                    20
SIGNBOARD                     10
Name: count, dtype: int64

In [185]:
filtered_df_PHYSCOND = master_df[master_df['PHYSICAL_CONDITION'] != 'UNKNOWN']
master_df['PHYSICAL_CONDITION'].value_counts()

PHYSICAL_CONDITION
NORMAL                          146951
UNKNOWN                          51633
REMOVED BY EMS                    3673
IMPAIRED - ALCOHOL                1807
OTHER                             1553
EMOTIONAL                         1246
FATIGUED/ASLEEP                    980
ILLNESS/FAINTED                    843
IMPAIRED - DRUGS                   282
HAD BEEN DRINKING                  281
IMPAIRED - ALCOHOL AND DRUGS       200
MEDICATED                           61
Name: count, dtype: int64

## Consideration

- There are 51,663 observations with "UNKNOWN" and 96,088 null values in the column.
- If "UNKNOWN" value is removed, the "NORMAL" physical condition has 146951, which accounts for 93% of the known value. Possible solution is to assign the mode, which is "NORMAL" to all the null values.



In [177]:
master_df['BAC_RESULT'].value_counts()

BAC_RESULT
TEST NOT OFFERED                   204553
TEST REFUSED                         2726
TEST PERFORMED, RESULTS UNKNOWN      1154
TEST TAKEN                            805
Name: count, dtype: int64

## Consideration

- There are 204553 observations where BAC test was not offered. It accounts for 97% of the observations. Possible solution is to use Mode and assign "TEST NOT OFFERED" to the null values.

In [175]:
X = master_df['AGE']
y = master_df['PRIM_CONTRIBUTORY_CAUSE']

np.corrcoef(X, y)

TypeError: unsupported operand type(s) for /: 'str' and 'int'

## .describe()

In [86]:
master_df.describe()

,CRASH_DATE,POSTED_SPEED_LIMIT,LANE_CNT,BEAT_OF_OCCURRENCE,NUM_UNITS,INJURIES_TOTAL,INJURIES_FATAL,INJURIES_INCAPACITATING,INJURIES_NON_INCAPACITATING,INJURIES_REPORTED_NOT_EVIDENT,INJURIES_NO_INDICATION,INJURIES_UNKNOWN,CRASH_HOUR,CRASH_DAY_OF_WEEK,CRASH_MONTH,NUM_PASSENGERS,VEHICLE_YEAR,TOTAL_VEHICLE_LENGTH,AGE,BAC_RESULT VALUE,YEAR
count,1858312,1.858312e+06,257.000000,1.858312e+06,1.858312e+06,1.858312e+06,1.858312e+06,1.858312e+06,1.858312e+06,1.858312e+06,1.858312e+06,1858312.0,1.858312e+06,1.858312e+06,1.858312e+06,457906.000000,1.563966e+06,2188.000000,1.303675e+06,2222.000000,1.858312e+06
mean,2022-01-05 09:01:05.914044928,2.909264e+01,2.785992,1.228066e+03,2.222681e+00,3.469261e-01,2.472674e-03,3.474874e-02,2.005180e-01,1.091867e-01,2.561203e+00,0.0,1.351261e+01,4.145358e+00,6.444955e+00,1.720595,2.014060e+03,52.589580,3.764151e+01,0.170536,2.021519e+03
min,2020-01-01 00:00:00,0.000000e+00,0.000000,1.110000e+02,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,1.000000e+00,1.000000e+00,1.000000,1.900000e+03,1.000000,-5.900000e+01,0.000000,2.020000e+03
25%,2021-02-06 17:02:00,3.000000e+01,2.000000,7.140000e+02,2.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.000000e+00,0.0,1.000000e+01,2.000000e+00,4.000000e+00,1.000000,2.008000e+03,40.000000,2.500000e+01,0.120000,2.021000e+03
50%,2022-01-11 17:57:00,3.000000e+01,2.000000,1.131000e+03,2.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.000000e+00,0.0,1.400000e+01,4.000000e+00,7.000000e+00,1.000000,2.014000e+03,53.000000,3.500000e+01,0.170000,2.022000e+03
75%,2022-12-20 07:00:00,3.000000e+01,4.000000,1.733000e+03,2.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,3.000000e+00,0.0,1.700000e+01,6.000000e+00,9.000000e+00,2.000000,2.018000e+03,65.000000,5.000000e+01,0.220000,2.022000e+03
max,2023-11-09 02:40:00,7.000000e+01,8.000000,6.100000e+03,1.800000e+01,2.100000e+01,3.000000e+00,1.000000e+01,1.900000e+01,1.100000e+01,4.800000e+01,0.0,2.300000e+01,7.000000e+00,1.200000e+01,46.000000,9.999000e+03,535.000000,1.100000e+02,1.000000,2.023000e+03
std,NaN,5.170034e+00,1.501600,6.962089e+02,7.008905e-01,8.930255e-01,5.634475e-02,2.444030e-01,6.709131e-01,4.744634e-01,1.831509e+00,0.0,5.545192e+00,2.002147e+00,3.276245e+00,1.790805,9.983012e+01,24.758989,1.703455e+01,0.105440,1.092123e+00


In [188]:
# Checking out the PRIN_CONTRIBUTORY_CAUSE column

master_df['PRIM_CONTRIBUTORY_CAUSE'].value_counts()

PRIM_CONTRIBUTORY_CAUSE
UNABLE TO DETERMINE                                                                 83621
FAILING TO YIELD RIGHT-OF-WAY                                                       53085
FOLLOWING TOO CLOSELY                                                               26314
FAILING TO REDUCE SPEED TO AVOID CRASH                                              26078
DISREGARDING TRAFFIC SIGNALS                                                        24689
IMPROPER TURNING/NO SIGNAL                                                          11340
IMPROPER OVERTAKING/PASSING                                                          9315
NOT APPLICABLE                                                                       9116
DISREGARDING STOP SIGN                                                               8654
OPERATING VEHICLE IN ERRATIC, RECKLESS, CARELESS, NEGLIGENT OR AGGRESSIVE MANNER     7296
DRIVING SKILLS/KNOWLEDGE/EXPERIENCE                                         

### Takeaway

- Most of the primary contributory cause could not be determined.
- There are currently 40 different classifications.

- Possible classification grouping: Human Factors, Speed and Aggressive Driving, Weather and Environmental Conditions, Vehicle-Related Factors, Traffic Violations, Road Design and Infrastructure, Inadequate Driver Training and Education, Medical Conditions, Wildlife and Environmental Factors, Construction Zones, Distractions Outside the Vehicle.

In [15]:
#master_df = master_df[master_df['PRIM_CONTRIBUTORY_CAUSE'] != 'UNABLE TO DETERMINE']
#master_df.info()